In [ ]:
# OS library
import sys
sys.path.append('../')

import glob
import pandas as pd
import numpy as np

# Magic to reload notebook
%load_ext autoreload
%autoreload 2
pd.options.display.max_columns = 100

# Covid Saúde

Painel de casos de doença pelo coronavírus 2019 (COVID-19) no Brasil pelo Ministério da Saúde ([website](https://covid.saude.gov.br/))

In [ ]:
# Do site exportam-se alguns arquivos .csv contendo as informações sobre o covid-19. É preciso concatená-los em 1 só dataframe
files = glob.glob("../data/raw/covid_saude/*.csv")
dfs = [pd.read_csv(f, sep=';') for f in files]

df_covid_saude = pd.concat(dfs)

In [ ]:
df_covid_saude_mun = df_covid_saude.loc[df_covid_saude['municipio'].notna()]

In [ ]:
df_covid_saude_mun = df_covid_saude_mun[['municipio', 'data', 'obitosNovos']]
df_covid_saude_mun.to_parquet('../data/app/covid_saude_obito_municipio.parquet', index=False)

# Vacinação

In [ ]:
df_caso = pd.read_csv('../data/raw/brasil_io/caso.csv')

In [ ]:
df_caso

In [ ]:
df_caso_full = pd.read_csv('../data/raw/brasil_io/caso_full.csv')

In [ ]:
df_caso_full

In [ ]:
df_vacinacao = pd.read_csv('../data/raw/part-00000-9bf0f6da-b13b-4025-9160-2dd9b163f3f3.c000.csv', sep=';')

In [ ]:
df_vacinacao.columns

In [ ]:
df_vacinacao

In [ ]:
df_vacinacao[['paciente_datanascimento','paciente_endereco_coibgemunicipio', 'estabelecimento_municipio_codigo', 'vacina_fabricante_nome', 'vacina_dataaplicacao', 'vacina_descricao_dose', 'vacina_nome']].to_parquet('../data/raw/vacinacao_acre.parquet', index=False)